In [7]:
import pandas as pd
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import gc
import numpy as np
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
import numpy as np
import lightgbm as lgb
import pandas as pd
import os 
from datetime import datetime
from Tuning_parameter import Lgb_params
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
import xgboost as xgb
import scipy

#from Tuning_parameter import Lgb_params


In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gc.collect()

1969

In [9]:
len(train.columns)

23

In [10]:
#id : 'bacno', 'cano', 'txkey'
#continuous : 'conam', 'iterm'
#time : 'locdt', 'loctm'
#binary : 'ecfg', 'insfg', 'ovrlt'
#multi category :'acqic', 'contp', 'csmcu', 'etymd',
#       'flbmk', 'flg_3dsmk', 'hcefg', 
#     'mcc', 'mchno', 'scity', 'stocn', 'stscd',


In [11]:
# Rand_under_sam = RandomUnderSampler(random_state=31)
# def sample(df):
#     Sampled_df = pd.DataFrame()
#     for i in range(1, 3):
#         Temp_df = df[df['Month'] == i]
#         Sample_X, Sample_y = Rand_under_sam.fit_resample(Temp_df[['txkey']], Temp_df['fraud_ind'])
#         Sample_X = pd.DataFrame(Sample_X, columns= ['txkey'])
#         Sample_y = pd.DataFrame(Sample_y, columns= ['fraud_ind'])
#         Sample_X = pd.merge(Sample_X, Temp_df, on ='txkey', how ='inner')
#         Sampled_df = pd.concat([Sampled_df, Sample_X], axis = 0)
#     return Sampled_df

In [12]:
def Process_loctm(df1, df2):
    def Str_turn_time(str1):
        str1 = str(int(str1))
        if len(str1) < 6:
            str1 = (6 - len(str1)) * '0' + str1
        return str1
    
    df_comb = pd.concat([df1, df2],axis=0)
    
    df_comb['Hour'] = df_comb['loctm'].apply(lambda x :Str_turn_time(x)[:2]).astype(int)
    df_comb['Morning'] = 0
    df_comb.loc[(df_comb['Hour'].astype('int') > 7) & (df_comb['Hour'].astype('int') < 22), 'Morning'] = 1
    
    df1 = df_comb[:len(df1)]
    df2 = df_comb[len(df1):]
    
    return df1, df2

def Separate_dt(df1, df2):
    df_comb = pd.concat([df1, df2],axis=0)
    
    df_comb['Month'] = 0
    df_comb.loc[df_comb['locdt'] < 121, 'Month'] = 4
    df_comb.loc[df_comb['locdt'] < 91, 'Month']  = 3
    df_comb.loc[df_comb['locdt'] < 61, 'Month']  = 2
    df_comb.loc[df_comb['locdt'] < 31, 'Month'] = 1
    
    df1 = df_comb[:len(df1)]
    df2 = df_comb[len(df1):]
    
    return df1, df2


# FREQUENCY ENCODE TOGETHER
def encode_FE(df1, df2, cols, replace=False):
    if replace:
        for col in cols:
            df = pd.concat([df1[col],df2[col]])
            vc = df.value_counts(dropna=True, normalize=True).to_dict()
            vc[-1] = -1
            df1[col] = df1[col].map(vc)
            df2[col] = df2[col].map(vc)
            print(col,', ',end='')
    else:
        for col in cols:
            df = pd.concat([df1[col],df2[col]])
            vc = df.value_counts(dropna=True, normalize=True).to_dict()
            vc[-1] = -1
            new_col = col+'_FE'
            df1[new_col] = df1[col].map(vc)
            df2[new_col] = df2[col].map(vc)
            print(new_col,', ',end='')
            
    return df1, df2


# LABEL ENCODE
def encode_LE(df1, df2, col):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)

    if df_comb.max()>32000: 
        train[col] = df_comb[:len(train)].astype('int32')
        test[col] = df_comb[len(train):].astype('int32')
    else:
        train[col] = df_comb[:len(train)].astype('int16')
        test[col] = df_comb[len(train):].astype('int16')
    
    return df1, df2

# COMBINE FEATURES
def encode_CB(df1, df2, col1, col2, replace = True):
    new_col = col1+'_'+col2
    df1[new_col] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[new_col] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    df1, df2 = encode_FE(df1, df2, [new_col], replace = replace)

    print(new_col,', ',end='')
    
    return df1, df2


def encode_AG(main_columns, uids, aggregations, df1, df2, fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            if main_column != col:
                for agg_type in aggregations:
                    new_col_name = main_column+'_'+col+'_'+agg_type
                    temp_df = pd.concat([df1[[col, main_column]], df2[[col,main_column]]])
                    
                    if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                    temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                            columns={agg_type: new_col_name})
    
                    temp_df.index = list(temp_df[col])
                    temp_df = temp_df[new_col_name].to_dict()   
    
                    df1[new_col_name] = df1[col].map(temp_df).astype('float32')
                    df2[new_col_name]  = df2[col].map(temp_df).astype('float32')
                    
                    if fillna:
                        df1[new_col_name].fillna(-1,inplace=True)
                        df2[new_col_name].fillna(-1,inplace=True)
                    
                    print("'"+new_col_name+"'",', ',end='')
    return df1, df2   

def Set_cate_cols(df1, df2, cols):
    for col in cols:
        df1[col] = df1[col].astype('category')
        df2[col] = df2[col].astype('category')
    return df1, df2

def Fill_na(df1, df2, cols):
    for col in cols:
        df1[col] = df1[col].fillna(-99)
        df2[col] = df2[col].fillna(-99)
    return df1, df2

def F1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'f1_score', f1_score(y_true, np.round(y_pred))

def get_mode(x):
    return scipy.stats.mode(x.values)[0][0]

less_cate_cols = ['contp', 'etymd', 'hcefg', 'stscd']
more_cate_cols = ['acqic', 'csmcu', 'mcc', 'mchno', 'scity', 'stocn']
#cate_cols = ['acqic', 'contp', 'csmcu', 'etymd', 'hcefg', 
#     'mcc', 'mchno', 'scity', 'stocn', 'stscd']

#FE
train['Mean_conam'] = (train['conam']+0.001) / (train['iterm'] +1)
test['Mean_conam']  = (test['conam']+0.001) / (test['iterm'] +1)


# train['Both_0'] = 0
train.loc[(train['conam'] == 0) & (train['iterm'] == 0), 'Both_0'] = 1
test['Both_0'] = 0
test.loc[(test['conam'] == 0) & (test['iterm'] == 0), 'Both_0'] = 1

print("Mean_conam", ',', end='')
train, test = Separate_dt(train, test)
#fill na
train, test = Fill_na(train, test, ['flbmk', 'flg_3dsmk'])

#Calculate time 
train, trst = Process_loctm(train, test)

train, test = encode_LE(train, test, 'insfg')
train, test = encode_LE(train, test, 'ecfg')
train, test = encode_LE(train, test, 'ovrlt')
train, test = encode_LE(train, test, 'flbmk')
train, test = encode_LE(train, test, 'flg_3dsmk')

#CB
train, test = encode_CB(train, test, 'stocn', 'scity', False)
train, test = encode_CB(train, test, 'stocn_scity', 'cano', True)
train, test = encode_CB(train, test, 'conam', 'iterm', True)
train, test = encode_CB(train, test, 'Month', 'bacno', True)
train, test = encode_CB(train, test, 'Month', 'cano', True)

#AG 
train, test = encode_AG(['conam', 'iterm', 'Mean_conam'], ['bacno', 'cano', 'Month_bacno', 'Month_cano'], ['mean', 'std', 'max', 'median', 'nunique', 'min', 'sum'], train, test)
train, test = encode_AG(['cano'], ['bacno'], ['nunique', 'count'], train, test)
train, test = encode_AG(['txkey'], ['bacno'], ['count'], train, test)
train, test = encode_AG(['txkey'], ['Month_bacno'], ['count'], train, test)
train, test = encode_AG(['txkey'], ['Month_cano'], ['count'], train, test)
train, test = encode_AG(['txkey'], ['cano'], ['count'], train, test)

#interection btw 'cano', 'bacno'
train['cnt_ratio_by_cano_bacno'] = train['txkey_cano_count'] / train['txkey_bacno_count'] 
test['cnt_ratio_by_cano_bacno'] = test['txkey_cano_count'] / test['txkey_bacno_count'] 

train['conam_ratio_by_cano_bacno'] = train['conam_cano_sum'] / train['conam_bacno_sum'] 
train['conam_ratio_by_bacno'] = train['conam'] / train['conam_bacno_sum'] 
train['conam_ratio_by_cano'] = train['conam'] / train['conam_cano_sum'] 

test['conam_ratio_by_cano_bacno'] = test['conam_cano_sum'] / test['conam_bacno_sum'] 
test['conam_ratio_by_bacno'] = test['conam'] / test['conam_bacno_sum'] 
test['conam_ratio_by_cano'] = test['conam'] / test['conam_cano_sum'] 


train, test = encode_FE(train, test, less_cate_cols, replace = False)
train, test = encode_FE(train, test, more_cate_cols, replace = True)

#set cate_feautre
train, test = Set_cate_cols(train, test, ['stscd', 'flg_3dsmk', 'flbmk', 'flg_3dsmk', 'contp', 'hcefg'])

train, test = encode_AG(more_cate_cols + ['stocn_scity'], ['bacno', 'cano'], ['nunique', 'count'], train, test)

# for col in ['stscd', 'flg_3dsmk', 'flbmk', 'flg_3dsmk', 'contp', 'hcefg']:
#     train[col +'_mode_gb_bacno'] = train.groupby('bacno')[col].transform('mode')
#     test[col +'_mode_gb_cano']  = test.groupby('cano')[col].transform('mode')


train.drop(['stocn_scity'], axis = 1, inplace = True)
test.drop(['stocn_scity'], axis = 1, inplace = True)


Mean_conam ,

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



stocn_scity_FE , stocn_scity , stocn_scity_cano , stocn_scity_cano , conam_iterm , conam_iterm , Month_bacno , Month_bacno , Month_cano , Month_cano , 'conam_bacno_mean' , 'conam_bacno_std' , 'conam_bacno_max' , 'conam_bacno_median' , 'conam_bacno_nunique' , 'conam_bacno_min' , 'conam_bacno_sum' , 'conam_cano_mean' , 'conam_cano_std' , 'conam_cano_max' , 'conam_cano_median' , 'conam_cano_nunique' , 'conam_cano_min' , 'conam_cano_sum' , 'conam_Month_bacno_mean' , 'conam_Month_bacno_std' , 'conam_Month_bacno_max' , 'conam_Month_bacno_median' , 'conam_Month_bacno_nunique' , 'conam_Month_bacno_min' , 'conam_Month_bacno_sum' , 'conam_Month_cano_mean' , 'conam_Month_cano_std' , 'conam_Month_cano_max' , 'conam_Month_cano_median' , 'conam_Month_cano_nunique' , 'conam_Month_cano_min' , 'conam_Month_cano_sum' , 'iterm_bacno_mean' , 'iterm_bacno_std' , 'iterm_bacno_max' , 'iterm_bacno_median' , 'iterm_bacno_nunique' , 'iterm_bacno_min' , 'iterm_bacno_sum' , 'iterm_cano_mean' , 'iterm_cano_std' , 

AttributeError: Cannot access callable attribute 'mode' of 'SeriesGroupBy' objects, try using the 'apply' method

In [13]:
for col in ['stscd', 'flg_3dsmk', 'flbmk', 'flg_3dsmk', 'contp', 'hcefg'] + ['Hour']:
    train[col +'_mode_gb_bacno'] = train.groupby('bacno')[col].transform(get_mode)
    test[col +'_mode_gb_cano']  = test.groupby('cano')[col].transform(get_mode)

In [20]:

#id : 'bacno', 'cano', 'txkey'
#continuous : 'conam', 'iterm'
#time : 'locdt', 'loctm'
#binary : 'ecfg', 'insfg', 'ovrlt'
#multi category :'acqic', 'contp', 'csmcu', 'etymd',
#       'flbmk', 'flg_3dsmk', 'hcefg', 
#     'mcc', 'mchno', 'scity', 'stocn', 'stscd',

SEED = 42
Excluded_cols = ['fraud_ind', 'txkey', 'Month', 'cano']
traning_cols = [col for col in train.columns if col not in Excluded_cols]
print('Total columns : ', len(traning_cols))

train_df = train[train.Month != 3]
val_df   = train[train.Month == 3]
test_df  = test

trn_data = lgb.Dataset(train_df[traning_cols], label=train_df['fraud_ind'])
val_data = lgb.Dataset(val_df[traning_cols], label=val_df['fraud_ind'])

Total columns :  160


In [21]:
Lgb_params = {
        'num_leaves': 32,
        'min_child_samples': 5,
        'objective': 'binary',
        'max_depth': -1,
        'learning_rate': 0.05,
        "boosting_type": "gbdt",
        #"subsample": 0.8,
        "bagging_seed": 11,
        #"verbosity": -1,
        'reg_alpha': 0.3,
        'reg_lambda': 0.3,
        #'colsample_bytree': 0.8,
        'num_boost_round' : 500, 
        'verbose_eval' : 100,
        'early_stopping_rounds' : 50,
        #'scale_pos_weight' : 700,
        'num_threads' :6,
        #'scale_pos_weight' :50
        }

In [22]:
Lgb = lgb.train(
            Lgb_params,
            trn_data,
            valid_sets = [trn_data, val_data],
            verbose_eval = 50,
            feval=F1_score, 
            evals_result={}
        )   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Training until validation scores don't improve for 50 rounds
[50]	training's binary_logloss: 0.0241269	training's f1: 0.648895	valid_1's binary_logloss: 0.0316941	valid_1's f1: 0.353643
[100]	training's binary_logloss: 0.0198374	training's f1: 0.720539	valid_1's binary_logloss: 0.0300651	valid_1's f1: 0.373037
[150]	training's binary_logloss: 0.0177965	training's f1: 0.754839	valid_1's binary_logloss: 0.0295321	valid_1's f1: 0.377416
Early stopping, best iteration is:
[123]	training's binary_logloss: 0.0187703	training's f1: 0.738762	valid_1's binary_logloss: 0.0297161	valid_1's f1: 0.382926


In [39]:
importance_df = pd.DataFrame()
importance_df['feature']    = traning_cols
importance_df['importance'] = np.log1p(Lgb.feature_importance(importance_type='gain', iteration=Lgb.best_iteration))

In [40]:
importance_df.head()

,feature,importance
0,Mean_conam,11.076542
1,acqic,10.107453
2,bacno,10.419316
3,conam,10.465919
4,contp,3.749843


In [ ]:
preds = np.round(Lgb.predict(test_df[traning_cols]))

In [ ]:
Xgb = xgb.XGBClassifier( 
        n_estimators=1000,
        max_depth=12, 
        learning_rate=0.5, 
        subsample=0.8,
        colsample_bytree=0.4, 
        #missing= 'NAN', 
        # USE CPU
        #nthread=4,
        #tree_method='hist' 
        # USE GPU
    )
Xgb.fit(train_df[traning_cols], train_df['fraud_ind'], 
        eval_set=[(val_df[traning_cols], val_df['fraud_ind'])],
        verbose=100, early_stopping_rounds=200, eval_metric = f1_eval)

In [ ]:
train.flbmk.dtypes

# Feature Selection

In [ ]:


Train_df      = Processed_data['Train_df']
Test_df       = Processed_data['Test_df']
Test_id       = Processed_data['Test_df']['txkey']
#Record_list   = Processed_data['Record_list']
Cate_feat     = Processed_data['Cate_feat']

N_folds       = Processed_data['N_folds']
Target        = 'fraud_ind'
#Time          = datetime.today().strftime('%m-%d-%H-%M')

def Cal_f1_score(True_y, Pre_y):
    return f1_score(True_y, Pre_y)
            
def F1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

def Split_group_kfolds():
    Train_X = Train_df.drop(['fraud_ind'], axis=1)
    print(Train_X['Month'].unique())
    Train_Y = Train_df['fraud_ind']
    Folds  = GroupKFold(n_splits=3)
    Splited_data = Folds.split(Train_X, Train_Y, groups=Train_X['Month']) 
    return Splited_data


clf = lgb.LGBMClassifier( 
                #early_stopping_rounds = 10, 
                #categorical_feature = Cate_feat, 
                #feval=F1_score, 
                #evals_result={},     
                num_leaves =  31,
                min_child_samples =  79,
                objective = 'binary',
                max_depth = -1,
                learning_rate =  0.025,
                boosting_type = 'gbdt',
                #"subsample": 0.8,
                bagging_seed = 42,
                #"verbosity": -1,
                #'reg_alpha': 0.3,
                #'reg_lambda': 0.3,
                #'colsample_bytree': 0.8,
                n_estimators = 1, 
                #verbose_eval = 100,
                #'scale_pos_weight' : 700,
                #num_threads  =4
                )

rfecv = RFECV(estimator=clf, step=1, cv = Split_group_kfolds(), verbose=2, n_jobs = -1, scoring = 'f1')

Exluded_cols = ['fraud_ind', 'txkey', 'Month']
All_Features = [Feat for Feat in Train_df.columns if Feat not in Exluded_cols]

rfecv.fit(Train_df[All_Features], Train_df[Target])
sel_features = [f for f, s in zip(All_Features, rfecv.support_) if s]
print('\n The selected features are {}:'.format(sel_features))
print(len(sel_features))

plt.figure(figsize=(12, 9))
plt.xlabel('Number of features ')
plt.ylabel('Cross-validation score (F1_score)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1) , rfecv.grid_scores_)
plt.show()

print((time.time() - Start_time)/60, 'minute')

In [ ]:
Xgb = xgb.XGBClassifier( 
        n_estimators=1000,
        max_depth=-1, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='f1_score',
        # USE CPU
        #nthread=4,
        #tree_method='hist' 
        # USE GPU
    )